In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
seed_type = os.listdir("/kaggle/input/plant-seedlings-classification/train")

In [ ]:
seed_type

In [ ]:
import cv2

In [ ]:
dim1 = []
dim2 = []
for image_filename in os.listdir("/kaggle/input/plant-seedlings-classification/train/Common Chickweed"):
 
 img = cv2.imread("/kaggle/input/plant-seedlings-classification/train/Common Chickweed"+"/"+image_filename)
 d1,d2,colors = img.shape
 dim1.append(d1)
 dim2.append(d2)

In [ ]:
np.mean(dim1)

In [ ]:
np.mean(dim2)

In [ ]:
image_shape=(230,230,3)
image_shape[:2]

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255, 
        rotation_range=360,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.5,
        vertical_flip=True,
        horizontal_flip=True,
        validation_split=0.2, 
    )

train_generator = image_gen.flow_from_directory(
        directory='/kaggle/input/plant-seedlings-classification/train',
        target_size=(230, 230),
        
        color_mode='rgb',
        class_mode="categorical",
        subset='training',
    )
validation_generator = image_gen.flow_from_directory(
        directory='/kaggle/input/plant-seedlings-classification/train',
        target_size=(230,230),
        
        color_mode='rgb',
        class_mode="categorical",
        subset='validation',
    )


In [ ]:


    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(
        directory='/kaggle/input/plant-seedlings-classification/',
        classes=['test'],
        target_size=(230, 230),
        batch_size=1,
        color_mode='rgb',
        shuffle=False,
        class_mode='categorical')

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten

In [ ]:
model=Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(12,activation='softmax'))
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop=EarlyStopping(monitor='val_loss',patience=10)

In [ ]:
batch_size=16

In [ ]:

train_generator.class_indices

In [ ]:
result=model.fit_generator(train_generator,epochs=44, #44 epochs
 validation_data=validation_generator,
 callbacks=[early_stop])

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
model.save("model1.h5")

In [ ]:
class_labels = list(train_generator.class_indices)
class_labels

In [ ]:
predictions = model.predict(test_generator, steps=test_generator.samples)

class_list = []

for i in range(0, predictions.shape[0]):
  y_class = predictions[i, :].argmax(axis=-1)
  class_list += [class_labels[y_class]]

submission = pd.DataFrame()
submission['file'] = test_generator.filenames
submission['file'] = submission['file'].str.replace(r'test/', '')
submission['species'] = class_list

submission.to_csv('submission.csv', index=False)